In [2]:
require(data.table)
require(cleanerR)

Loading required package: data.table
Loading required package: cleanerR


Reading the files

In [3]:
#setwd("..") #If it cant see data uncomment this line, it may already be inside data
#list.files()
setwd("Data")
files=list.files(pattern=".csv")
Kickstarter=lapply(files,fread)
Kickstarter=Reduce(rbind,Kickstarter)
cat("The dimension of the Kickstarter dataset is",dim(Kickstarter),"\n")
cat("Its attributes are:",names(Kickstarter),"\n")

The dimension of the Kickstarter dataset is 205091 37 
Its attributes are: backers_count blurb category converted_pledged_amount country created_at creator currency currency_symbol currency_trailing_code current_currency deadline disable_communication friends fx_rate goal id is_backing is_starrable is_starred launched_at location name permissions photo pledged profile slug source_url spotlight staff_pick state state_changed_at static_usd_rate urls usd_pledged usd_type 


Since cleanerR works best when predicting categorical data lets see the variables of interest

In [10]:
sapply(Kickstarter,class)

backers_count                    blurb                 category 
               "integer"              "character"              "character" 
converted_pledged_amount                  country               created_at 
               "integer"              "character"                "integer" 
                 creator                 currency          currency_symbol 
             "character"              "character"              "character" 
  currency_trailing_code         current_currency                 deadline 
               "logical"              "character"                "integer" 
   disable_communication                  friends                  fx_rate 
               "logical"              "character"                "numeric" 
                    goal                       id               is_backing 
               "numeric"                "integer"                "logical" 
            is_starrable               is_starred              launched_at 
               "logical"                "logical"                "integer" 
                location                     name              permissions 
             "character"              "character"              "character" 
                   photo                  pledged                  profile 
             "character"                "numeric"              "character" 
                    slug               source_url                spotlight 
             "character"              "character"                "logical" 
              staff_pick                    state         state_changed_at 
               "logical"              "character"                "integer" 
         static_usd_rate                     urls              usd_pledged 
               "numeric"              "character"                "numeric" 
                usd_type 
             "character"

In [11]:
sapply(sapply(Kickstarter,unique),length)

backers_count                    blurb                 category 
                    3191                   184680                      169 
converted_pledged_amount                  country               created_at 
                   31086                       22                   186053 
                 creator                 currency          currency_symbol 
                  204004                       14                        6 
  currency_trailing_code         current_currency                 deadline 
                       2                        5                   174027 
   disable_communication                  friends                  fx_rate 
                       2                        3                       51 
                    goal                       id               is_backing 
                    5120                   186153                        3 
            is_starrable               is_starred              launched_at 
                       2                        3                   186028 
                location                     name              permissions 
                   15280                   185586                        3 
                   photo                  pledged                  profile 
                  186155                    45242                   186155 
                    slug               source_url                spotlight 
                  186153                      169                        2 
              staff_pick                    state         state_changed_at 
                       2                        5                   176382 
         static_usd_rate                     urls              usd_pledged 
                    9434                   186295                    73446 
                usd_type 
                       3

Lets order the variables that are not numeric by their unique frequency

In [20]:
caracteres=which(!(sapply(Kickstarter,class) %in% c('numeric','integer')))
KickCar=Kickstarter[,..caracteres]
ClassesFrequency=sapply(sapply(KickCar,unique),length)
ClassesFrequency[order(as.numeric(ClassesFrequency))]

currency_trailing_code  disable_communication           is_starrable 
                     2                      2                      2 
             spotlight             staff_pick                friends 
                     2                      2                      3 
            is_backing             is_starred            permissions 
                     3                      3                      3 
              usd_type       current_currency                  state 
                     3                      5                      5 
       currency_symbol               currency                country 
                     6                     14                     22 
              category             source_url               location 
                   169                    169                  15280 
                 blurb                   name                   slug 
                184680                 185586                 186153 
                 photo                profile                   urls 
                186155                 186155                 186295 
               creator 
                204004

Now we will do the following test, try to predict spotlight, state, currency, country ,category , source_url and location, the variables with more unique variables are close to primary keys of the dataset

The process will be the following we shall make a copy of the dataset, randomly set a percentage of the values of a atribute to NA, and then try to predict then,after predicted we compare the predicted vs the original to determine accuracy, while also showing what would be the accuracy using the distribution of the original data, the percentages we shall work with is 5%,10%,25% and 50% of missing data.

First lets define the function for this test:

In [4]:
test=function(k2,perc,goal,row){
    vec=c()
    k2=data.frame(k2)
    for(i in 1:10){
        k3=k2
        Positions=sample(1:nrow(k3),perc*nrow(k3))
        k3[Positions,goal]=NA
        k3=CompleteDataset(k3,row,goal)
        vec[i]=sum(k3[Positions,goal]==k2[Positions,goal])/length(Positions)

    }
    return(vec)
}

To start doing the tests

In [5]:
SpotLight=which(names(Kickstarter)=="spotlight")
State=which(names(Kickstarter)=="state")
Currency=which(names(Kickstarter)=="currency")
Country=which(names(Kickstarter)=="country")
Category=which(names(Kickstarter)=="category")
SourceUrl=which(names(Kickstarter)=="source_url")

In [43]:
cat("Spotlight statitical accuracy is:",sum((as.numeric(table(Kickstarter$spotlight))/nrow(Kickstarter))^2),"\n" )
cat("State statitical accuracy is:",sum((as.numeric(table(Kickstarter$state))/nrow(Kickstarter))^2),"\n" )
cat("Currency statitical accuracy is:",sum((as.numeric(table(Kickstarter$currency))/nrow(Kickstarter))^2),"\n" )
cat("Country statitical accuracy is:",sum((as.numeric(table(Kickstarter$country))/nrow(Kickstarter))^2),"\n" )
cat("Category statitical accuracy is:",sum((as.numeric(table(Kickstarter$category))/nrow(Kickstarter))^2),"\n" )
cat("SourceUrl statitical accuracy is:",sum((as.numeric(table(Kickstarter$source_url))/nrow(Kickstarter))^2),"\n" )

Spotlight statitical accuracy is: 0.5061778 
State statitical accuracy is: 0.4462562 
Currency statitical accuracy is: 0.6307156 
Country statitical accuracy is: 0.6291397 
Category statitical accuracy is: 0.009811976 
SourceUrl statitical accuracy is: 0.00928841 


In [35]:
rows=BestVector(df = Kickstarter,goal = SpotLight,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,SpotLight,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,SpotLight,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,SpotLight,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,SpotLight,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")

[1] 1
[1] 2
[1] 3
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 


In [36]:
rows=BestVector(df = Kickstarter,goal = State,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,State,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,State,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,State,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,State,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")

[1] 1
[1] 2
[1] 3


Warning message in names(df)[solution] == names(df_aux):
“comprimento do objeto maior não é múltiplo do comprimento do objeto menor”

The test accuracy mean is: 0.871377 Its standart deviation is: 0.00377913 
The test accuracy mean is: 0.8700375 Its standart deviation is: 0.001589842 
The test accuracy mean is: 0.8706741 Its standart deviation is: 0.0008510364 
The test accuracy mean is: 0.8702336 Its standart deviation is: 0.0007285919 


In [37]:
rows=BestVector(df = Kickstarter,goal = Currency,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,Currency,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,Currency,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,Currency,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,Currency,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")

[1] 1
[1] 2
[1] 3
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 
The test accuracy mean is: 1 Its standart deviation is: 0 


In [38]:
rows=BestVector(df = Kickstarter,goal = Country,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,Country,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,Country,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,Country,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,Country,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")

[1] 1
[1] 2
[1] 3
The test accuracy mean is: 0.9643164 Its standart deviation is: 0.001102819 
The test accuracy mean is: 0.9634843 Its standart deviation is: 0.00119732 
The test accuracy mean is: 0.9635318 Its standart deviation is: 0.0003870088 
The test accuracy mean is: 0.963537 Its standart deviation is: 0.0003161103 


In [39]:
rows=BestVector(df = Kickstarter,goal = Category,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,Category,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,Category,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,Category,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,Category,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")

[1] 1
[1] 2
[1] 3
The test accuracy mean is: 0.9007802 Its standart deviation is: 0.002496621 
The test accuracy mean is: 0.899293 Its standart deviation is: 0.00218737 
The test accuracy mean is: 0.899569 Its standart deviation is: 0.0008822136 
The test accuracy mean is: 0.8997221 Its standart deviation is: 0.0005949762 


In [ ]:
rows=BestVector(df = Kickstarter,goal = SourceUrl,maxi = 3,repetitions = nrow(Kickstarter),trigger = 0.9,ratio=0.9)
vec=test(Kickstarter,0.05,SourceUrl,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.10,SourceUrl,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.25,SourceUrl,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")
vec=test(Kickstarter,0.5,SourceUrl,rows)
cat("The test accuracy mean is:",mean(vec),"Its standart deviation is:",sd(vec),"\n")